In [ ]:
'''相關庫'''
import pandas as pd
import numpy as np
import semopy as sem

'''模擬數據'''
# 設定隨機種子以確保結果的可重現性
np.random.seed(42)

# 定義變量
variables = [
    "work1", "work2", "work3",
    "happy", "mood1", "mood2",
    "perform1", "perform2",
    "approval", "beliefs",
    "selftalk", "imagery"
]

# 生成模擬的變異數-共變異數矩陣
cov_matrix_simulated = np.random.rand(len(variables), len(variables))
# 使矩陣對稱
cov_matrix_simulated = (cov_matrix_simulated + cov_matrix_simulated.T) / 2
# 將對角線設置為1，表示變量自身的變異數
np.fill_diagonal(cov_matrix_simulated, 1)

# 將模擬的變異數-共變異數矩陣轉換為DataFrame，以符合後續分析的要求
dataframe = pd.DataFrame(
    data=cov_matrix_simulated, index=variables, columns=variables
)

'''SEM'''
# 定義SEM模型
mod = """
# measurement model
Construc =~ beliefs + selftalk + imagery
Dysfunc =~ perform1 + perform2 + approval
WellBe =~ happy + mood1 + mood2
JobSat =~ work1 + work2 + work3
# error covariance
happy ~~ mood2 
# structural part
Dysfunc ~ Construc
WellBe ~ Construc + Dysfunc
JobSat ~ Construc + Dysfunc + WellBe
"""

# 擬合SEM模型
model = sem.Model(mod)
res = model.fit(obj="MLW", solver="SLSQP", cov=dataframe, n_samples=263)

# 獲取和顯示估計結果
estimates = model.inspect(std_est=True)
stats = sem.calc_stats(model)
basic_stats = stats[["DoF", "chi2", "RMSEA", "NFI", "TLI", "CFI", "GFI"]]

# 打印基本統計量
print(basic_stats)

# 或打印出詳細的分析報表
print(stats)

# 繪製模型結構圖，將生成的圖像保存為"model_structure.png"
sem.semplot(model, "model_structure.png")